In [3]:
import numpy as np
import uproot as ur
import ROOT as rt
import root_numpy as rtnp
import ROOT.RooFit as rf
from scipy.special import erf
import sys
sys.path.append('../lib')
from histo_utilities import create_TH1D, create_TH2D, std_color_list
from cebefo_style import Set_2D_colz_graphics

import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 1
CMS_lumi.extraText = "     Preliminary"

donotdelete = []

In [4]:
# %jsroot on

In [5]:
from analysis_utilities import drawOnCMSCanvas, extarct, extarct_multiple, createSel

In [6]:
rt.gErrorIgnoreLevel = rt.kError
rt.RooMsgService.instance().setGlobalKillBelow(rt.RooFit.ERROR)

# Data loading

In [10]:
tMC = ur.open('../data/cmsMC_private/BPH_Tag-B0_MuNuDmst-pD0bar-kp_13TeV-pythia8_SoftQCD_PTFilter5_0p0-evtgen_HQET2_central_PU35_10-2-3_v0/B02DstMu_candidates/out_0.root')['outA']['Tevts']
print tMC.keys(), '\n\n'
branches = tMC.keys()
for b in np.sort(branches):
    print b

['LumiBlock', 'MC_B_P', 'MC_B_eta', 'MC_B_phi', 'MC_B_pt', 'MC_D0_P', 'MC_D0_eta', 'MC_D0_phi', 'MC_D0_pt', 'MC_Dst_P', 'MC_Dst_eta', 'MC_Dst_phi', 'MC_Dst_pt', 'MC_Est_mu', 'MC_M2_miss', 'MC_mu_P', 'MC_mu_eta', 'MC_mu_phi', 'MC_mu_pt', 'MC_pis_P', 'MC_pis_eta', 'MC_pis_phi', 'MC_pis_pt', 'MC_q2', 'Run', 'eventNumber', 'n_B', 'n_D0', 'n_Dst', 'n_K', 'n_pi', 'n_pis', 'trgMu_charge', 'trgMu_eta', 'trgMu_phi', 'trgMu_pt', 'B_E', 'B_P', 'B_eta', 'B_phi', 'B_pt', 'B_pz', 'D0_E', 'D0_P', 'D0_eta', 'D0_phi', 'D0_pt', 'D0_pz', 'Dst_E', 'Dst_P', 'Dst_eta', 'Dst_phi', 'Dst_pt', 'Dst_pz', 'Est_mu', 'K_E', 'K_P', 'K_eta', 'K_phi', 'K_pt', 'K_pz', 'M2_miss', 'chi2_D0pis', 'chi2_MuDst', 'chi2_kpi', 'cos_D0pis_vtxMu', 'cos_MuDst_vtxBest', 'cos_kpi_vtxMu', 'd_vtxD0pis_vtxMu', 'd_vtxkpi_vtxMu', 'dca_D0pis_vtxMu', 'dca_kpi_vtxMu', 'mass_D0pis', 'mass_MuDst', 'mass_kpi', 'mu_E', 'mu_P', 'mu_eta', 'mu_phi', 'mu_pt', 'mu_pz', 'pi_E', 'pi_P', 'pi_eta', 'pi_phi', 'pi_pt', 'pi_pz', 'pis_E', 'pis_P', 'pis_eta'

In [13]:
file_loc = {}
file_loc['data'] = '../data/cmsRD/Run2018D/B02DstMu_19*_BPH*.root'
file_loc['mu'] = '../data/cmsMC_private/BPH_Tag-B0_MuNuDmst-pD0bar-kp_13TeV-pythia8_SoftQCD_PTFilter5_0p0-evtgen_HQET2_central_PU35_10-2-3_v0/B02DstMu_candidates/out*.root'
file_loc['tau'] = '../data/cmsMC_private/BPH_Tag-B0_TauNuDmst-pD0bar-kp-t2mnn_pythia8_SoftQCD_PTFilter5_0p0-evtgen_HQET2_central_PU35_10-2-3_v0/B02DstMu_candidates/out*.root'

In [14]:
dSet = {}
for k, fn in file_loc.iteritems():
    dSet[k] = extarct_multiple(fn, branches=branches, flag=k)
    print k, ':', dSet[k]['Est_mu'].shape[0], 'candidates'

mu : 15963 candidates
tau : 7222 candidates
data : 323091 candidates


# Analysis parameters

Selection declaration

In [ ]:
analysis_selection = {}
analysis_selection['mass_MuDst'] = ['<', 5.280]
analysis_selection['cos_MuDst_vtxMu'] = ['>', 0.98]
# analysis_selection['M2_miss'] = ['>', -2]

Binning declaration

In [ ]:
binning = {}
binning['q2'] = [4, -2, 10.7]
binning['M2_miss'] = [40/2, -5, 10]
binning['Est_mu'] = [30/2, 0.10, 2.500]

# Create histograms

In [ ]:
histo= {}
for k in file_loc.keys():
    histo[k] = rt.TH3F('h_'+k, k, 
                     binning['q2'][0], binning['q2'][1], binning['q2'][2],
                     binning['M2_miss'][0], binning['M2_miss'][1], binning['M2_miss'][2],      
                     binning['Est_mu'][0], binning['Est_mu'][1], binning['Est_mu'][2]
                    )

    d = dSet[k]
    sel = createSel(d, analysis_selection)
    rtnp.fill_hist(histo[k], np.column_stack((d['q2'][sel], d['M2_miss'][sel], d['Est_mu'][sel])))

In [ ]:
# tf = rt.TFile('../data/_root/histos_3D.root', 'recreate')
# for k,v in histo.iteritems():
#     v.Write()
# tf.Close()

# Fit just $q^2$

In [ ]:
h_q2 = {}
bin_q2_only = [50, -5, 12]
leg = rt.TLegend(0.5,0.6,0.9,0.9)
for k in file_loc.keys():
    d = dSet[k]
    sel = createSel(d, analysis_selection)
    
    h_q2[k] = create_TH1D(d['q2'][sel], name='h_q2_'+k, binning=bin_q2_only)
    h_q2[k].Sumw2()

In [ ]:
if True:
    tf = rt.TFile('../data/_root/histos_1Dq2.root', 'recreate')
    for k,v in h_q2.iteritems():
        if k == 'mu':
            v.Scale(h_q2['data'].GetEntries()*0.99/v.GetEntries())
        if k == 'tau':
            v.Scale(h_q2['data'].GetEntries()*0.01/v.GetEntries())
        v.Write()
    tf.Close()

In [ ]:
q2 = rt.RooRealVar('q2', 'q^{2}', bin_q2_only[1], bin_q2_only[2], 'GeV^{2}')

data = rt.RooDataHist('data', 'Data', rt.RooArgList(q2), h_q2['data'])

dh_mu = rt.RooDataHist('dh_mu', 'mu', rt.RooArgList(q2), h_q2['mu'])
pdf_mu = rt.RooHistPdf('pdf_mu', 'pdf_mu', rt.RooArgSet(q2), dh_mu)
n_mu = rt.RooRealVar('n_mu', 'N_{#mu}', 0.99*h_q2['data'].GetEntries(), 100, 1.1*h_q2['data'].GetEntries())
pdf_mu_ext = rt.RooExtendPdf('e_mu', 'e_mu', pdf_mu, n_mu)


dh_tau = rt.RooDataHist('dh_tau', 'tau', rt.RooArgList(q2), h_q2['tau'])
pdf_tau = rt.RooHistPdf('pdf_tau', 'pdf_tau', rt.RooArgSet(q2), dh_tau)
n_tau = rt.RooRealVar('n_tau', 'N_{#tau}', 0.1*h_q2['data'].GetEntries(), 100, 1.1*h_q2['data'].GetEntries())
pdf_tau_ext = rt.RooExtendPdf('e_tau', 'e_tau', pdf_tau, n_tau)

pdf_tot = rt.RooAddPdf('pdftot', 'pdftot', rt.RooArgList(pdf_mu_ext, pdf_tau_ext))
pdf_tot.fitTo(data)

In [ ]:
frame = q2.frame(rf.Title(""))
data.plotOn(frame, rf.MarkerStyle(1))

pdf_tot.plotOn(frame, rf.Components('e_mu, e_tau'), rf.LineColor(4), rf.LineWidth(2), rf.Name('Mu'))
pdf_tot.plotOn(frame, rf.Components('e_tau'), rf.LineColor(2), rf.LineWidth(2), rf.Name('Tau'))
        
# i_pad = i_q2*N_cols + i_v + 1
# pad = canvas.cd(i_pad)
# pad.SetTopMargin(0.05)
# pad.SetBottomMargin(0.13)
# pad.SetRightMargin(0.03)
# pad.SetLeftMargin(0.22)
    
CMS_lumi.extraText = "     Preliminary"
c = drawOnCMSCanvas(CMS_lumi, frame)
# c.SetLogy()

# Create RooFit workspace

In [ ]:
q2 = rt.RooRealVar('q2', 'q^{2}', binning['q2'][1], binning['q2'][2], 'GeV^{2}')
M2_miss = rt.RooRealVar('M2_miss', 'M^{2}_{miss}', binning['M2_miss'][1], binning['M2_miss'][2], 'GeV^{2}')
Est_mu = rt.RooRealVar('Est_mu', 'E_{#mu}*', binning['Est_mu'][1], binning['Est_mu'][2], 'GeV')

In [ ]:
data = rt.RooDataHist('data', 'Data', rt.RooArgList(q2, M2_miss, Est_mu), histo['data'])

In [ ]:
dh_mu = rt.RooDataHist('dh_mu', 'mu', rt.RooArgList(q2, M2_miss, Est_mu), histo['mu'])
pdf_mu = rt.RooHistPdf('pdf_mu', 'pdf_mu', rt.RooArgSet(q2, M2_miss, Est_mu), dh_mu)

n_mu = rt.RooRealVar('n_mu', 'N_{#mu}', 0.99*histo['data'].GetEntries(), 0, 1.05*histo['data'].GetEntries())
pdf_mu_ext = rt.RooExtendPdf('e_mu', 'e_mu', pdf_mu, n_mu)

In [ ]:
dh_tau = rt.RooDataHist('dh_tau', 'tau', rt.RooArgList(q2, M2_miss, Est_mu), histo['tau'])
pdf_tau = rt.RooHistPdf('pdf_tau', 'pdf_tau', rt.RooArgSet(q2, M2_miss, Est_mu), dh_tau)

n_tau = rt.RooRealVar('n_tau', 'N_{#tau}', 0.1*histo['data'].GetEntries(), 0, histo['data'].GetEntries())
pdf_tau_ext = rt.RooExtendPdf('e_tau', 'e_tau', pdf_tau, n_tau)

In [ ]:
pdf_tot = rt.RooAddPdf('pdftot', 'pdftot', rt.RooArgList(pdf_mu_ext, pdf_tau_ext))
pdf_tot.fitTo(data)

In [ ]:
print 'N data =', histo['data'].GetEntries()
print 'N mu = {:.0f} +/- {:.0f}'.format(n_mu.getVal(), n_mu.getError())
print 'N tau = {:.0f} +/- {:.0f}'.format(n_tau.getVal(), n_tau.getError())

# Plot the results

In [ ]:
N_cols = len(binning.keys())-1
N_rows = binning['q2'][0]
canvas = rt.TCanvas('c_out', 'c_out', 50, 50, N_cols*400, 400*N_rows)
canvas.SetTickx(0)
canvas.SetTicky(0)
canvas.Divide(N_cols, N_rows)

CMS_lumi.extraText = "        Preliminary"

In [ ]:
q2_edges = np.linspace(binning['q2'][1], binning['q2'][2], 1+binning['q2'][0], True)

for i_q2 in range(binning['q2'][0]):
    q2_l = q2_edges[i_q2]
    q2_h = q2_edges[i_q2+1]

    q2_txt = '{:.2f} <  q^{{2}}  < {:.2f} GeV^{{2}}'.format(q2_l, q2_h)
    q2_cut = 'q2 > {:.2f} &&  q2 < {:.2f}'.format(q2_l, q2_h)

    for i_v, var in enumerate([M2_miss, Est_mu]):
        frame = var.frame(rf.Title(""))
        aux_data = data.reduce(q2_cut)
        aux_data.plotOn(frame, rf.MarkerStyle(1), rf.Name('Data'))
#         data.plotOn(frame, rf.Cut(q2_cut), rf.MarkerStyle(1))s
        
#         pdf_tot.plotOn(frame, rf.LineColor(15), rf.LineWidth(2), rf.LineStyle(9))
        pdf_tot.plotOn(frame, rf.Components('e_mu, e_tau'), rf.Cut(q2_cut), rf.LineColor(4), rf.LineWidth(2), rf.Name('Mu'))
        pdf_tot.plotOn(frame, rf.Components('e_tau'), rf.Cut(q2_cut), rf.LineColor(2), rf.LineWidth(2), rf.Name('Tau'))
        
        i_pad = i_q2*N_cols + i_v + 1
        pad = canvas.cd(i_pad)
        pad.SetTopMargin(0.05)
        pad.SetBottomMargin(0.13)
        pad.SetRightMargin(0.03)
        pad.SetLeftMargin(0.22)
        
        frame.Draw()
        CMS_lumi.CMS_lumi(pad, -1, 0, 0.75*1.3, 0.6*1.3)
        
        frame.SetMaximum(1.2*frame.GetMaximum())
        frame.GetYaxis().SetTitleOffset(1.7)
        
        l = rt.TLatex()
        l.SetTextAlign(11)
        l.SetTextSize(0.055)
        l.SetTextFont(42)
        l.DrawLatexNDC(0.26, 0.87, q2_txt)

        if i_pad == 1:
            leg = rt.TLegend(0.6, 0.6, 0.93, 0.82)
            leg.SetTextFont(42)
            leg.SetTextAlign(12)
            leg.SetLineWidth(1)
            leg.SetBorderSize(1)
            leg.AddEntry('Data', 'Data', 'lep')
            leg.AddEntry('Mu', 'B#rightarrowD*#mu#nu', 'l')
            leg.AddEntry('Tau', 'B#rightarrowD*#tau#nu', 'l')
            leg.Draw()

canvas.Draw()

In [ ]:
canvas.SaveAs('/Users/olmo/Desktop/RDst_fit.png')